In [ ]:
import torch 
from torch import nn 
from torchvision import transforms as T 
import numpy as np
from tqdm import tqdm

from timm import create_model
import torch

model = create_model('resnet10t', in_chans=1, norm_layer=lambda chans: torch.nn.GroupNorm(num_groups=8, num_channels=chans), num_classes=2)

WEIGHTS_PATH = '/ssd005/projects/exactvu_pca/checkpoint_store/2023-12-05_timm_resnet10.ckpt'

model.load_state_dict(torch.load(WEIGHTS_PATH)['model'])
model.eval()
model.cuda()


In [ ]:

from medAI.datasets import ExactNCT2013BmodeImagesWithAutomaticProstateSegmentation, ExactNCT2013BmodePatches, CohortSelectionOptions, PatchOptions, ExactNCT2013RFImagePatches, ExactNCT2013RFImagesWithAutomaticProstateSegmentation

core_ds = ExactNCT2013BmodeImagesWithAutomaticProstateSegmentation('test', cohort_selection_options=CohortSelectionOptions())
core_ds_rf = ExactNCT2013RFImagesWithAutomaticProstateSegmentation('test', cohort_selection_options=CohortSelectionOptions())

class ImageDataset: 
    def __init__(self, transform=None) -> None:
        self.bmode_images = core_ds
        self.rf_images = core_ds_rf

        self.transform = transform

    def __getitem__(self, idx): 
        out = self.bmode_images[idx]
        out.update(self.rf_images[idx])
        if self.transform is not None: 
            out = self.transform(out)
        return out
    
    def __len__(self): 
        return len(self.bmode_images)
    
core_ds = ImageDataset()



In [ ]:
def core_ds_transform(item): 
    image = item['bmode']
    image = torch.from_numpy(image).float()
    image = image.unsqueeze(0)
    H, W = image.shape[-2:]
    item['bmode'] = image

    image = item['rf_image']
    image = torch.from_numpy(image).float()
    image = image.unsqueeze(0)
    item['rf_image'] = image

    mask = item['prostate_mask']
    mask = torch.from_numpy(mask).float()
    mask = mask.unsqueeze(0)
    mask = T.Resize((H, W), interpolation=T.InterpolationMode.NEAREST)(mask)
    item['prostate_mask'] = mask

    mask = item['needle_mask']
    mask = torch.from_numpy(mask).float()
    mask = mask.unsqueeze(0)
    mask = T.Resize((H, W), interpolation=T.InterpolationMode.NEAREST)(mask)
    item['needle_mask'] = mask
    return item


def default_patch_transform(patch):
    patch = torch.from_numpy(patch).float()
    patch = (patch - patch.min()) / (patch.max() - patch.min())
    patch = patch.unsqueeze(0)
    from torchvision import transforms as T
    patch = T.Resize((256, 256), antialias=True)(patch)

    return patch


def compute_patch_size_pixels(patch_size_mm, image_size_pixels, image_size_mm):
    pH, pW = patch_size_mm
    H, W = image_size_pixels
    H_mm, W_mm = image_size_mm
    return int(pH * H / H_mm), int(pW * W / W_mm)



In [ ]:
core_ds.transform = core_ds_transform




In [ ]:
from einops import rearrange, repeat

def n_windows(image_size, patch_size, stride, padding=(0, 0)):
    H, W = image_size
    pH, pW = patch_size
    sH, sW = stride
    padH, padW = padding
    nH = (H + 2 * padH - pH) // sH + 1
    nW = (W + 2 * padW - pW) // sW + 1
    return nH, nW


def compute_batched_output(model, input, batch_size): 
    n = input.shape[0]
    output = []
    for i in range(0, n, batch_size):
        output.append(model(input[i:i+batch_size]))
    output = torch.cat(output, dim=0)
    return output


def create_heatmap(model, image, patch_size_mm=(5,5), stride_mm=(1,1), image_size_mm=(28, 46.06), device='cuda', compute_batch_size=128):

    model = model.to(device)
    image = image.to(device)

    H, W = image.shape[-2:]

    pH, pW = compute_patch_size_pixels(patch_size_mm, (H, W), image_size_mm)
    sH, sW = compute_patch_size_pixels(stride_mm, (H, W), image_size_mm)
    padding = pH // 2, pW // 2
    nH, nW = n_windows((H, W), (pH, pW), (sH, sW), padding=padding)
   
    unfold = nn.Unfold(kernel_size=(pH, pW), stride=(sH, sW), padding=padding)
    patches = unfold(image)
    patches = rearrange(patches, 'b (c k1 k2) (nh nw) -> (b nh nw) c k1 k2', c=1, k1=pH, k2=pW, nh=nH, nw=nW)
    patches = T.Resize((256, 256), antialias=True)(patches)

    patches_min = patches.min(-1, keepdim=True)[0].min(-2, keepdim=True)[0]
    patches_max = patches.max(-1, keepdim=True)[0].max(-2, keepdim=True)[0]

    patches = (patches - patches_min) / (patches_max - patches_min)
    predictions = model(patches)
    probs = predictions.softmax(dim=-1)[:, [1]]
    probs = repeat(probs, 'b c -> b c k1 k2', c=1, k1=pH, k2=pW)
    probs = rearrange(probs, '(b nh nw) c k1 k2 -> b (c k1 k2) (nh nw)', nh=nH, nw=nW, k1=pH, k2=pW)

    fold = nn.Fold(output_size=(H, W), kernel_size=(pH, pW), stride=(sH, sW), padding=padding)
    n_contributions = fold(torch.ones_like(probs))
    hmap = fold(probs) / n_contributions 

    return hmap 
        

def create_heatmap_v2(model, image): 
    image = image.numpy()[0][0]
    from medAI.datasets.nct2013 import compute_base_positions, PatchOptions, compute_mask_intersections, select_patch
    base_positions = list(compute_base_positions((28, 46.06), patch_options=PatchOptions()))
    # base_positions = list(compute_mask_intersections(base_positions, prostate_mask, 'prostate', (28, 46.06), threshold=0.9))

    heatmap = np.zeros_like(image)
    n_contributions = np.zeros_like(image)

    patches = []
    for base_position in base_positions:
        patch, _ = select_patch(image, base_position, patch_options=PatchOptions())
        patches.append(default_patch_transform(patch))
    patches = torch.stack(patches)
    patches = patches.cuda()

    with torch.no_grad():
        logits = model(patches.cuda())
        probs = torch.softmax(logits, dim=1)
        probs = probs[:, 1]

    for base_position, prob in zip(base_positions, probs):
        patch, _ = select_patch(heatmap, base_position, patch_options=PatchOptions())
        patch[:] += prob.cpu().numpy()
        patch, _ = select_patch(n_contributions, base_position, patch_options=PatchOptions())
        patch[:] += 1

    hmap = heatmap / n_contributions
    hmap = torch.from_numpy(hmap).float().unsqueeze(0).unsqueeze(0)
    return hmap 


def get_contours(mask):

    import cv2
    mask_uint8 = (mask[0].detach().cpu().numpy() * 255).astype('uint8')
    contours = cv2.findContours(mask_uint8, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    mask_contours = np.zeros_like(mask_uint8)
    cv2.drawContours(mask_contours, contours[0], -1, 255, 6)

    return mask_contours


In [ ]:
TARGET_DIR = '/scratch/ssd004/scratch/pwilson/2023-12-06_heatmaps_v0'


# saving the raw heatmaps
model.eval()
os.makedirs(os.path.join(TARGET_DIR, 'raw'), exist_ok=True)

i = 0
for item in tqdm(core_ds):
    i += 1
    rf_image = item['rf_image'].unsqueeze(0)
    image = item['bmode'].unsqueeze(0)
    tag = item['tag']
    grade = item['grade']
    prostate_mask = item['prostate_mask']
    needle_mask = item['needle_mask']
    pct_cancer = item['pct_cancer']
    patient_tag = item['case_id']

    if os.path.exists(os.path.join(TARGET_DIR, 'raw', f'{tag}.npy')):
        continue

    # generate heatmap
    patch_size_mm = (5, 5)
    stride_mm = (1, 1)
    image_size_mm = (28, 46.06)
    device = 'cuda'
    hmap = create_heatmap_v2(model, rf_image)
    from skimage.filters import gaussian
    # hmap = torch.tensor(hmap).float().unsqueeze(0).unsqueeze(0)
    hmap = torch.nn.functional.interpolate(hmap, size=(image.shape[-2], image.shape[-1]), mode='bilinear', align_corners=False)
    import matplotlib.pyplot as plt
    hmap = hmap[0, 0].detach().cpu().numpy()
    hmap = gaussian(hmap, sigma=10)

    np.save(os.path.join(TARGET_DIR, 'raw', f'{tag}.npy'), hmap)




In [ ]:
from skimage.filters import gaussian
import matplotlib.pyplot as plt

HEATMAP_VERSION = 'v1'


i = 0 
for item in tqdm(core_ds):
    i += 1
    rf_image = item['rf_image'].unsqueeze(0)
    image = item['bmode'].unsqueeze(0)
    tag = item['tag']
    grade = item['grade']
    prostate_mask = item['prostate_mask']
    needle_mask = item['needle_mask']
    pct_cancer = item['pct_cancer']
    patient_tag = item['case_id']

    fname = f'{tag}_{grade}.png'
    fname = os.path.join(TARGET_DIR, HEATMAP_VERSION, patient_tag, fname)
    if os.path.exists(fname):
        continue

    hmap = np.load(os.path.join(TARGET_DIR, 'raw', f'{tag}.npy'))

    #  display and save
    cmap_array = np.load('/ssd005/projects/exactvu_pca/bmode_learning_data/G7.npy')
    cmap_array = cmap_array / 255
    from matplotlib.colors import ListedColormap, LinearSegmentedColormap
    cmap = ListedColormap(cmap_array)

    # for ai, color should be red for cancer, blue for benign, transparent for 0.5
    #ai_cmap = LinearSegmentedColormap.from_list('ai_cmap', [(0, 0, 1), (0, 0, 0, 0), (1, 0, 0)])
    ai_cmap = 'jet'
    #ai_cmap = 'seismic'
    ai_base_alpha = 0.3
    alpha = ai_base_alpha * np.ones_like(hmap)
    # alpha = alpha * (hmap > 0.2)
    #mps = np.max(np.stack([hmap, 1-hmap]), axis=0)
    #conf = (mps - 0.5) * 2
    # alpha = alpha * conf

    prostate_mask = gaussian(prostate_mask[0].cpu().numpy(), sigma=15)
    prostate_mask = (prostate_mask - prostate_mask.min()) / (prostate_mask.max() - prostate_mask.min())
    alpha = alpha * prostate_mask

    fig, ax = plt.subplots(2, 1, figsize=(4, 5))

    ax[0].imshow(np.flip(image[0, 0].detach().cpu().numpy().astype('uint8'), axis=0), cmap=cmap)
    ax[0].imshow(np.flip(get_contours(needle_mask), axis=0), alpha=np.flip(get_contours(needle_mask), axis=0) / 255)
    ax[0].axis('off')

    ax[1].imshow(np.flip(image[0, 0].detach().cpu().numpy().astype('uint8'), axis=0), cmap=cmap)
    ax[1].imshow(np.flip(hmap, axis=0), cmap=ai_cmap, alpha=np.flip(alpha, axis=0), vmin=0, vmax=1)
    ax[1].imshow(np.flip(get_contours(needle_mask), axis=0), alpha=np.flip(get_contours(needle_mask), axis=0) / 255)
    ax[1].axis('off')
    title = f'{tag} - grade {grade}' 
    if not np.isnan(pct_cancer): 
        title += f'\n involvement {pct_cancer:.2f}%'
    fig.suptitle(title)
    fig.tight_layout()
    os.makedirs(os.path.dirname(fname), exist_ok=True)
    plt.savefig(fname, dpi=300)
    
    # if i > 5: 
    #     break
    plt.close()
    



In [ ]:
from skimage.filters import gaussian
import matplotlib.pyplot as plt
import pandas as pd

outputs = pd.DataFrame()

i = 0 
for item in tqdm(core_ds):
    i += 1
    rf_image = item.pop('rf_image').unsqueeze(0)
    image = item.pop('bmode').unsqueeze(0)
    needle_mask = item.pop('needle_mask')
    prostate_mask = item.pop('prostate_mask')
    tag = item['tag']

    row = item.copy()

    hmap = np.load(os.path.join(TARGET_DIR, 'raw', f'{tag}.npy'))

    # compute average prediction inside needle 
    needle_mask_ = needle_mask[0].detach().cpu().numpy()
    hmap_flat = hmap.flatten()
    needle_mask_flat = needle_mask_.flatten()
    hmap_valid = hmap_flat[needle_mask_flat > 0.5]
    hmap_valid = hmap_valid[~np.isnan(hmap_valid)]  # remove NaNs
    avg_pred = hmap_valid.mean()
    row['avg_hmap_value_needle_region'] = avg_pred

    # compute average prediction inside prostate
    prostate_mask_ = prostate_mask[0].detach().cpu().numpy()
    hmap_flat = hmap.flatten()
    prostate_mask_flat = prostate_mask_.flatten()
    hmap_valid = hmap_flat[prostate_mask_flat > 0.5]
    hmap_valid = hmap_valid[~np.isnan(hmap_valid)]  # remove NaNs
    avg_pred = hmap_valid.mean()
    row['avg_hmap_value_prostate_region'] = avg_pred

    outputs = pd.concat([outputs, pd.DataFrame(row, index=[i])])




In [ ]:
outputs_query = outputs.query('grade != "GS7"')
roc_auc_score(outputs_query['grade'] != 'Benign', outputs_query['avg_hmap_value_needle_region'])


In [ ]:
plt.hist(outputs['avg_hmap_value_needle_region'][outputs['grade'] == 'Benign'], bins=100, alpha=0.5, label='benign')
plt.hist(outputs['avg_hmap_value_needle_region'][outputs['grade'] != 'Benign'], bins=100, alpha=0.5, label='cancer')
plt.legend()
auc = roc_auc_score(outputs['grade'] != 'Benign', outputs['avg_hmap_value_needle_region'])
plt.title('Average heatmap value in needle region - AUROC {:.3f}'.format(auc))
plt.xlabel('Average heatmap value')
plt.ylabel('Count')


plt.figure()
plt.hist(outputs['avg_hmap_value_prostate_region'][outputs['grade'] == 'Benign'], bins=100, alpha=0.5, label='benign')
plt.hist(outputs['avg_hmap_value_prostate_region'][outputs['grade'] != 'Benign'], bins=100, alpha=0.5, label='cancer')
plt.legend()
auc = roc_auc_score(outputs['grade'] != 'Benign', outputs['avg_hmap_value_prostate_region'])
plt.title('Average heatmap value in prostate region - AUROC {:.3f}'.format(auc))
plt.xlabel('Average heatmap value')
plt.ylabel('Count')





In [ ]:
outputs_q = outputs.query('grade != "Benign"')
plt.scatter(outputs_q['pct_cancer'], outputs_q['avg_hmap_value_needle_region'] * 100, alpha=0.5)


# get best fit line
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(outputs_q['pct_cancer'].values.reshape(-1, 1), outputs_q['avg_hmap_value_needle_region'].values.reshape(-1, 1) * 100)
x = np.linspace(0, 100, 1000)
y = lr.predict(x.reshape(-1, 1))
plt.plot(x, y, color='red', label=f'best fit line - slope {lr.coef_[0][0]:.3f}')
plt.legend()
plt.xlabel('Percent cancer involvement')
plt.ylabel('Average heatmap value in needle region (%)')
plt.title('Average heatmap value vs. percent involvement for cancer cores')



In [ ]:
df_patient = outputs.groupby('case_id').avg_hmap_value_needle_region.mean().reset_index()
df_patient['grade'] = outputs.groupby('case_id').grade.first().values
df_patient['avg_hmap_value_prostate_region'] = outputs.groupby('case_id').avg_hmap_value_prostate_region.mean().values


In [ ]:
roc_auc_score(outputs['grade'] != 'Benign', outputs['avg_hmap_value_needle_region'])


In [ ]:
from skimage.filters import gaussian
import matplotlib.pyplot as plt

core_predictions = []
core_labels = []
core_involvement = []

i = 0 
for item in tqdm(core_ds):
    i += 1
    rf_image = item['rf_image'].unsqueeze(0)
    image = item['bmode'].unsqueeze(0)
    tag = item['tag']
    grade = item['grade']
    prostate_mask = item['prostate_mask']
    needle_mask = item['needle_mask']
    pct_cancer = item['pct_cancer']
    patient_tag = item['case_id']

    hmap = np.load(os.path.join(TARGET_DIR, 'raw', f'{tag}.npy'))

    # compute average prediction inside needle 
    needle_mask_ = needle_mask[0].detach().cpu().numpy()
    hmap_flat = hmap.flatten()
    needle_mask_flat = needle_mask_.flatten()
    hmap_valid = hmap_flat[needle_mask_flat > 0.5]
    hmap_valid = hmap_valid[~np.isnan(hmap_valid)]  # remove NaNs
    avg_pred = hmap_valid.mean()
    #print(avg_pred)

    core_predictions.append(avg_pred)
    core_labels.append(int(grade != "Benign"))
    core_involvement.append(pct_cancer if not np.isnan(pct_cancer) else 0)

core_predictions = np.array(core_predictions)
core_labels = np.array(core_labels)
core_involvement = np.array(core_involvement)

from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve, average_precision_score
print("All cores auroc", roc_auc_score(core_labels, core_predictions))
print("High involvement auroc", roc_auc_score(core_labels[core_involvement > 40], core_predictions[core_involvement > 40]))

#roc_auc_score(core_labels[np.logical_or(core_involvement == 0, core_involvement > 40)], core_predictions[np.logical_or(core_involvement == 0, core_involvement > 40)])
plt.hist(core_predictions[core_labels == 0], bins=100, alpha=0.5, label='benign')
plt.hist(core_predictions[core_labels == 1], bins=100, alpha=0.5, label='cancer')
plt.legend()


In [ ]:
core_predictions = np.array(core_predictions)
core_labels = np.array(core_labels)
core_involvement = np.array(core_involvement)

from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve, average_precision_score
roc_auc_score(core_labels, core_predictions)
#roc_auc_score(core_labels[np.logical_or(core_involvement == 0, core_involvement > 40)], core_predictions[np.logical_or(core_involvement == 0, core_involvement > 40)])


In [ ]:
plt.hist(core_predictions[core_labels == 0], bins=100, alpha=0.5, label='benign')
plt.hist(core_predictions[core_labels == 1], bins=100, alpha=0.5, label='cancer')
plt.legend()


In [ ]:
np.logical_or(core_involvement == 0, core_involvement > 40)


In [ ]:
core_predictions = np.array(core_predictions)
core_labels = np.array(core_labels)
core_involvement = np.array(core_involvement)



In [ ]:
ds_test = ExactNCT2013RFImagePatches('test', cohort_selection_options=CohortSelectionOptions(), patch_options=PatchOptions(prostate_mask_threshold=.9, needle_mask_threshold=0.6))
ds_val = ExactNCT2013RFImagePatches('val', cohort_selection_options=CohortSelectionOptions(), patch_options=PatchOptions(prostate_mask_threshold=.9, needle_mask_threshold=0.6))

def default_patch_transform(patch):
    patch = torch.from_numpy(patch).float()
    patch = (patch - patch.min()) / (patch.max() - patch.min())
    patch = patch.unsqueeze(0)
    from torchvision import transforms as T
    patch = T.Resize((256, 256), antialias=True)(patch)

    return patch

def transform(item): 
    patch = item['patch']
    patch = default_patch_transform(patch)
    label = torch.tensor(item['grade'] != "Benign").long()
    core_id = item['id']
    core_id = torch.tensor(core_id).long()
    return patch, label, core_id

ds_val.transform = transform
ds_test.transform = transform

model = model.cuda()
model.eval()


def run_test_epoch(model, dataset):

    labels = [] 
    ids = []
    prob_cancer = []
    all_logits = []

    for batch in tqdm(torch.utils.data.DataLoader(dataset, batch_size=256, shuffle=False, num_workers=4)):
        x, y, id = batch
        x = x.cuda()
        y = y.cuda()
        with torch.no_grad():
            logits = model(x)
            probs = torch.softmax(logits, dim=1)

        prob_cancer.append(probs[:, 1].cpu())
        all_logits.append(logits.cpu())
        labels.append(y.cpu())
        ids.append(id.cpu())

    labels = torch.cat(labels)
    ids = torch.cat(ids)
    prob_cancer = torch.cat(prob_cancer)
    logits = torch.cat(all_logits)

    return labels, ids, prob_cancer, logits


val_outputs = run_test_epoch(model, ds_val)
test_outputs = run_test_epoch(model, ds_test)



In [ ]:
labels, ids, prob_cancer, logits = test_outputs

core_labels = []
core_prob_cancer = []
for i in ids.unique():
    idx = ids == i
    core_labels.append(labels[idx].max())
    core_prob_cancer.append((prob_cancer[idx]).float().mean())
core_labels = torch.stack(core_labels)
core_prob_cancer = torch.stack(core_prob_cancer)

from sklearn.metrics import roc_auc_score, recall_score
THRESHOLD=0.42
metrics = {}
metrics['auc'] = roc_auc_score(core_labels, core_prob_cancer)
metrics['sensitivity'] = recall_score(core_labels, core_prob_cancer > THRESHOLD)
metrics['speicificity'] = recall_score(core_labels == 0, core_prob_cancer < THRESHOLD)
print(metrics)
import matplotlib.pyplot as plt
plt.hist(core_prob_cancer[core_labels == 0], bins=20, alpha=0.5, label='benign')
plt.hist(core_prob_cancer[core_labels == 1], bins=20, alpha=0.5, label='cancer')
plt.axvline(THRESHOLD, color='k', linestyle='--')
plt.legend()




In [ ]:
hmap = create_heatmap(ds_test.dataset[0]['rf_image'], ds_test.dataset[0]['prostate_mask'], model)
ds_bmode = ExactNCT2013BmodeImagesWithAutomaticProstateSegmentation('test', cohort_selection_options=CohortSelectionOptions())
plt.imshow(hmap, extent=[0, 46.06, 28, 0], vmin=0, vmax=1, cmap='seismic')


In [ ]:
ds_test.base_positions

In [ ]:
plt.hist(prob_cancer[labels == 0].cpu().numpy(), bins=50, alpha=0.5, label='benign')
plt.hist(prob_cancer[labels == 1].cpu().numpy(), bins=50, alpha=0.5, label='cancer')


In [ ]:
layer = torch.nn.Linear(2, 2)
optimizer = torch.optim.LBFGS(layer.parameters(), lr=0.1)

logits = logits.cuda()
labels = labels.cuda()
layer = layer.cuda()

def closure():
    out_logits = layer(logits)
    loss = torch.nn.functional.cross_entropy(out_logits, labels, weight=torch.tensor([1/(labels == 0).sum(), 1/(labels == 1).sum()], device='cuda'))
    optimizer.zero_grad()
    loss.backward()
    return loss

optimizer.step(closure)




In [ ]:
out_logits = layer(logits).detach().cpu()
labels = labels.cpu()
out_probs = torch.softmax(out_logits, dim=1)[:, 1]
plt.hist(out_probs[labels == 0].cpu().numpy(), bins=50, alpha=0.5, label='benign')
plt.hist(out_probs[labels == 1].cpu().numpy(), bins=50, alpha=0.5, label='cancer')

core_prob_cancer = []
core_labels = []
for i in ids.unique():
    idx = ids == i
    core_prob_cancer.append(out_probs[idx].mean())
    core_labels.append(labels[idx].max())
core_prob_cancer = torch.stack(core_prob_cancer)
core_labels = torch.stack(core_labels)

roc_auc_score(core_labels, core_prob_cancer)

plt.figure()
plt.hist(core_prob_cancer[core_labels == 0].cpu().numpy(), bins=50, alpha=0.5, label='benign')
plt.hist(core_prob_cancer[core_labels == 1].cpu().numpy(), bins=50, alpha=0.5, label='cancer')



In [ ]:
plt.hist(core_prob_cancer[core_labels == 0].numpy(), bins=20, alpha=0.5, label='benign')
plt.hist(core_prob_cancer[core_labels == 1].numpy(), bins=20, alpha=0.5, label='cancer')


In [ ]:
corewise_prob_cancer = []
corewise_labels = []
for core in np.unique(ids):
    core_mask = ids == core
    core_prob_cancer = prob_cancer[core_mask]
    core_labels = labels[core_mask]
    core_prob_cancer = core_prob_cancer.mean()
    core_labels = core_labels[0]
    corewise_prob_cancer.append(core_prob_cancer)
    corewise_labels.append(core_labels)

corewise_prob_cancer = np.array(corewise_prob_cancer)
corewise_labels = np.array(corewise_labels)

corewise_prob_cancer 
from sklearn.metrics import roc_auc_score
roc_auc_score(corewise_labels, corewise_prob_cancer)
ds.dataset.dataset.core_info.patient_id.unique()
